## Load libraries and necessary data files

In [11]:
library(data.tree)
library(data.table)
load('../data/mutation_type_pd.rda')

In [12]:
names(pdfinal) <- c('P_driver','P_passenger','P_null')
pdfinal

P_driver P_passenger      P_null 
0.001094647 0.018984124 0.979921230

In [34]:
birthrate <- function(node, type, sd, sp) {
        nd <- 0; np <- 0
        switch(type,
            { b <- ((1+sd)^(node$nd+1))/((1+sp)^(node$np  )); nd <- 1 },
            { b <- ((1+sd)^(node$nd  ))/((1+sp)^(node$np+1)); np <- 1 },
            { b <- node$B }
        )
            
        return(list(b,nd,np))
            
}

In [35]:

maybeTranspose <- function(node, sd, sp) {

    if (node$B==0){ # if the birth rate of the clone is zero, skip the node
        return()
    }
    
    # increase the number of cells by the existing number * the birth rate
    nc <- tail(node$ncells,n=1) + round(tail(node$ncells,n=1)*node$B)
    
    # sample from binomial distribution for number of transpositions
    if (nc < 4.2e9) {ntrans <- rbinom(1,nc,mu)} # rbinom() fails for large n
    else {ntrans <- nc*mu} # If n is too large, use the expected number of events (mean of distribution)
    if (ntrans > 0) {
        
        nc <- nc-ntrans
        types <- sample(sample(1:-1,ntrans,replace=TRUE,prob=pdfinal)) # sample mutation types from pdfinal
        for (i in 1:ntrans) {
            l<<-l+1
            tmp <- birthrate(node,types[i],sd,sp)
            node$AddChild(l, ncells=1, B=tmp[[1]], nd=node$nd+tmp[[2]], np=node$np+tmp[[3]])
        }
        
    }   
    node$ncells <- append(node$ncells,nc)

}

### Define parameters

In [72]:
#--- Set simulation parameters
N0 <- 1           # Initial number of cells in root clone
mu <- 0.2         # Probability of transposition / timestep of a single cell

NT <- 16           # Number of time steps

### Clone tree creation

In [74]:
l<-1 # Clone counter

N <- N0 # total population size

CellPop <- Node$new(1) # Initialize data.tree as single node

CellPop$ncells <- c(N0) # Set initial number of cells of clone
CellPop$B <-  1         # Set initial birth rate of clone
CellPop$np <- 0         # Set initial number of drivier mutations
CellPop$nd <- 0         # Set initial number of passenger mutations

ptm <- proc.time()
for (i in 2:NT) {

    t <- Traverse(CellPop,traversal='pre-order',filterFun=function(x) tail(x$ncells,n=1) > 0)    
    lapply(t,maybeTranspose,0.1,0.001)

    if (CellPop$totalCount==1) {
        N <- append(N, tail(CellPop$ncells,n=1))
    } else {
        N <- append(N,sum(vapply(CellPop$Get('ncells'),tail,n=1L,FUN.VALUE = numeric(1))))
    }
    
}
(proc.time() - ptm)[3]

print(CellPop,'ncells','B')
# sum(vapply(CellPop$Get('ncells'),tail,n=1L,FUN.VALUE = numeric(1)))
# CellPop$Get('ncells')
# N

elapsed 
  6.002

                                    levelName
1   1                                        
2    ¦--2                                    
3    ¦   ¦--7                                
4    ¦   ¦   ¦--20                           
5    ¦   ¦   ¦   ¦--40                       
6    ¦   ¦   ¦   ¦   ¦--129                  
7    ¦   ¦   ¦   ¦   ¦   ¦--497              
8    ¦   ¦   ¦   ¦   ¦   ¦   ¦--1789         
9    ¦   ¦   ¦   ¦   ¦   ¦   ¦   ¦--7014     
10   ¦   ¦   ¦   ¦   ¦   ¦   ¦   °--7015     
11   ¦   ¦   ¦   ¦   ¦   ¦   ¦--1790         
12   ¦   ¦   ¦   ¦   ¦   ¦   ¦   ¦--3547     
13   ¦   ¦   ¦   ¦   ¦   ¦   ¦   °--3548     
14   ¦   ¦   ¦   ¦   ¦   ¦   ¦--3546         
15   ¦   ¦   ¦   ¦   ¦   ¦   ¦--7012         
16   ¦   ¦   ¦   ¦   ¦   ¦   °--7013         
17   ¦   ¦   ¦   ¦   ¦   ¦--930              
18   ¦   ¦   ¦   ¦   ¦   ¦   ¦--1791         
19   ¦   ¦   ¦   ¦   ¦   ¦   ¦   ¦--3550     
20   ¦   ¦   ¦   ¦   ¦   ¦   ¦   ¦   °--7018 
21   ¦   ¦   ¦   ¦   ¦   ¦   ¦   °